## **Load Data from Google Drive**

##### **Libraries**

In [1]:
import os
import pandas as pd

##### **Connect Google Drive to Google Colaborative Notebook**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##### **Change current directory to the given path**

In [4]:
PATH_TO_FOLDER = '/content/drive/My Drive/TELECOM_PARIS - MS_BGD/Advanced_machine_learning'
os.chdir(PATH_TO_FOLDER)

##### **Import Data**

In [9]:
%%time
train_data = pd.read_csv('cruise_ship_info.csv')

CPU times: user 6.19 ms, sys: 946 µs, total: 7.14 ms
Wall time: 641 ms


In [10]:
print(train_data.shape)

(158, 9)


## **Réseau de neurones avec TPU**

#### **Source**

Source du programme initial :
https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/classification_iris_data_with_keras.ipynb#scrollTo=IgQge6h7AqDw

In [49]:
import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras

In [50]:
print(tf.__version__)
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/classification_iris_data_with_keras.ipynb')

2.3.0


###**Data, model and training**

####**Resolve TPU addresses**



In [51]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

####**FLAGS used as model params**



In [52]:
# Model specific parameters

# TPU address
tpu_address = TF_MASTER

# Number of epochs
epochs = 50

# Number of steps_per_epoch
steps_per_epoch = 5

# NOTE: Total number of training steps = Number of epochs * Number of steps_per_epochs

####**Download training input data and define prediction input & output**

In [53]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

PREDICTION_INPUT_DATA = {
    'SepalLength': [6.9, 5.1, 5.9, 6.0, 5.5, 6.2, 5.5, 6.3],
    'SepalWidth': [3.1, 3.3, 3.0, 3.4, 2.5, 2.9, 4.2, 2.8],
    'PetalLength': [5.4, 1.7, 4.2, 4.5, 4.0, 4.3, 1.4, 5.1],
    'PetalWidth': [2.1, 0.5, 1.5, 1.6, 1.3, 1.3, 0.2, 1.5],
}

PREDICTION_OUTPUT_DATA = ['Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

####**Define a Keras model**

In [54]:
def get_model():
  return keras.Sequential([
    keras.layers.Dense(10, input_shape=(4,), activation=tf.nn.relu, name = "Dense_1"),
    keras.layers.Dense(10, activation=tf.nn.relu, name = "Dense_2"),
    keras.layers.Dense(3, activation=None, name = "logits"),
    keras.layers.Dense(3, activation=tf.nn.softmax, name = "softmax")
  ])

####**Compiling the model with a distribution strategy**

In [65]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.77.202.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.77.202.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [56]:
with strategy.scope():
  model = get_model()
  model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1), 
                loss=tf.keras.losses.sparse_categorical_crossentropy, 
                metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
Dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
logits (Dense)               (None, 3)                 33        
_________________________________________________________________
softmax (Dense)              (None, 3)                 12        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


####**Train the model on TPU**

In [57]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Train the model
model.fit(
  train_x.values, train_y.values,
  steps_per_epoch = steps_per_epoch,
  epochs=epochs,
)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Epoch 1/50
1/5 [=====>........................] - ETA: 1s - loss: 1.2840 - accuracy: 0.3333WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0017s vs `on_train_batch_end` time: 0.0103s). Check your callbacks.


5/5 [==============================] - 0s 70ms/step - loss: 1.1065 - accuracy: 0.2917
Epoch 2/50
5/5 [==============================] - 0s 11ms/step - loss: 0.7887 - accuracy: 0.6167
Epoch 3/50
5/5 [==============================] - 0s 11ms/step - loss: 0.5876 - accuracy: 0.7333
Epoch 4/50
5/5 [==============================] - 0s 12ms/step - loss: 0.6192 - accuracy: 0.7083
Epoch 5/50
5/5 [==============================] - 0s 12ms/step - loss: 0.3727 - accuracy: 0.8833
Epoch 6/50
5/5 [==============================] - 0s 11ms/step - loss: 0.3624 - accuracy: 0.8417
Epoch 7/50
5/5 [==============================] - 0s 11ms/step - loss: 0.3464 - accuracy: 0.8750
Epoch 8/50
5/5 [==============================] - 0s 11ms/step - loss: 0.3232 - accuracy: 0.8667
Epoch 9/50
5/5 [==============================] - 0s 11ms/step - loss: 0.3823 - accuracy: 0.8583
Epoch 10/50
5/5 [==============================] - 0s 11ms/step - loss: 0.2626 - accuracy: 0.9333
Epoch 11/50
5/5 [=======================

####**Evaluation of the model**

In [58]:
model.evaluate(test_x.values, test_y.values,
    batch_size=8)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


1/4 [======>.......................] - ETA: 1s - loss: 0.1874 - accuracy: 0.8750WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_test_batch_end` time: 0.0142s). Check your callbacks.


4/4 [==============================] - 0s 118ms/step - loss: 0.0703 - accuracy: 0.9667


[0.07026984542608261, 0.9666666984558105]

####**Save the model**

In [62]:
model.save_weights('./DNN_TPU_1024.h5', overwrite=True)

###**Prediction**

####**Prediction Data**

In [59]:
COLUMNS_NAME=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.DataFrame(PREDICTION_INPUT_DATA, columns=COLUMNS_NAME)
print(data)

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.9         3.1          5.4         2.1
1          5.1         3.3          1.7         0.5
2          5.9         3.0          4.2         1.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3
5          6.2         2.9          4.3         1.3
6          5.5         4.2          1.4         0.2
7          6.3         2.8          5.1         1.5


####**Prediction on TPU**

In [60]:
predictions = model.predict(data.values.astype(np.float32))
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))


Prediction is "Virginica" (83.7%), expected "Virginica"

Prediction is "Setosa" (99.7%), expected "Setosa"

Prediction is "Versicolor" (99.1%), expected "Versicolor"

Prediction is "Versicolor" (98.2%), expected "Versicolor"

Prediction is "Versicolor" (98.4%), expected "Versicolor"

Prediction is "Versicolor" (99.7%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (71.8%), expected "Virginica"


####**Prediction on CPU**

In [63]:
cpu_model = get_model()
cpu_model.load_weights('./DNN_TPU_1024.h5')
cpu_predictions = cpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(cpu_predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))


Prediction is "Virginica" (83.8%), expected "Virginica"

Prediction is "Setosa" (99.7%), expected "Setosa"

Prediction is "Versicolor" (99.1%), expected "Versicolor"

Prediction is "Versicolor" (98.2%), expected "Versicolor"

Prediction is "Versicolor" (98.4%), expected "Versicolor"

Prediction is "Versicolor" (99.7%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (70.8%), expected "Virginica"
